# Section I: Building the Initial Configuration

The first step to simulating a lipid bilayer is generating the initial configuration. Here the `mBuild` package is used to generate a minimal stratum corneum system containing an equimolar mixture of ceramide N-hydroxy sphingosine, cholesterol, and free fatty acids. The `mBuild` software allows for the easy and reproducible construction of a preassembled bilayer with tunable parameters. This script will use the following packages:
`{insert URLs for each of these}`
- `numpy` 
- `mbuild`
- `mdtraj`
- `py3dmol`

These packages are installed using `conda` in the cell below. The Anaconda software can be installed from `{insert URL}`

In [ ]:
!conda install -y numpy 
!conda install -y mbuild 
!conda install -y mdtraj
!conda install -y py3dmol

The required modules are imported below. The `bilayer` module can be found at `./building/bilayer.py`. This contains a `python` class used to construct a bilayer `mBuild` `Compound`.

In [ ]:
import mbuild as mb
import numpy as np
import mdtraj as md
from bilayer import Bilayer

The configurations of each molecule type are saved as `.mol2` files. Here these compounds are loaded from disk and saved to a dictionary. 

In [ ]:
def load_molecule(filename):
    """Worker function to load the configuration of a single lipid."""
    lipid = mb.load(filename)
    lipid.translate_to([0, 0, 0])
    return lipid

prototypes = dict()

for molecule_name in ["cer", "chol", "ffa", "tip3p"]:
    filename = f"./molecules/{molecule_name}.mol2"
    molecule = load_molecule(filename)
    molecule.name = molecule_name
    prototypes.update({molecule_name : molecule})

Below are the parameters chosen for the system being simulated. The parameter space (for example, the lipid composition and/or water content) can be easily explored further by adjusting these values.

In [ ]:
# Equimolar ratio of CER, CHOL, and FFA
lipids = [(prototypes["cer"], 0.33),
          (prototypes["chol"], 0.33),
          (prototypes["ffa"], 0.34)]

n_lipids_per_edge = 6

tilt_angle = 10 * np.pi / 180.0 # radians
area_per_lipid = .32 # nm^2
spacing_z = 2.8 #nm

waters_per_lipid = 40
water_density = 1.0 # g/cm^3
water_mass = 18.01 # amu

The bilayer system is set up using the parameters set above and saved as a GROMACS `.gro` file. 

In [ ]:
system = Bilayer(lipids,
                 ref_atoms=[77, 2, 26],
                 n_lipids_x=n_lipids_per_edge, 
                 n_lipids_y=n_lipids_per_edge,
                 area_per_lipid=area_per_lipid,
                 spacing_z=spacing_z,
                 solvent=prototypes["tip3p"], 
                 solvent_per_lipid=waters_per_lipid,
                 solvent_density=water_density, 
                 solvent_mass=water_mass,
                 tilt=tilt_angle,
                 random_seed=2019,
                 mirror=False)

# Create box with 0.1 nm boundary
box = mb.Box(mins=[0, 0, 0],
             maxs=(system.solvent_components.boundingbox.lengths + np.array([0.05, 0.05, 0.05])))

# Translate to box center
system.translate_to(box.lengths * 0.5)

# Convert to mdTraj Trajectory and save to disk
configuration = system.to_trajectory(residues=["cer", "chol", "ffa", "tip3p"], box=box)
configuration.save("start.gro")

A render of the system can visually inspected using the `py3dmol` renderer built into `mBuild` in order to validate the configuration.

In [ ]:
# visualize
system.visualize()

When finished, exit this notebook window and return to the main notebook.